## **Libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import librosa
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
!pip install numpy==1.26.4 --quiet
!pip install jams librosa --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 52.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 7.4 MB/s eta 0:00:00


In [ ]:
import numpy as np

In [ ]:
import jams

In [ ]:
BASE_PATH = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet"

AUDIO_DIR = os.path.join(BASE_PATH, "audio", "mono_mic")
ANNOTATION_DIR = os.path.join(BASE_PATH, "annotation")

## **Extract Note Events & Mel Spectrogram**

In [ ]:
def extract_note_events(jams_path):
    jam = jams.load(jams_path)
    note_events = []

    for ann in jam.search(namespace='note_midi'):
        for note in ann:
            note_events.append({
                'onset': note.time,
                'duration': note.duration,
                'pitch': note.value
            })

    return pd.DataFrame(note_events)

In [ ]:
def extract_melspec(audio_path, sr=16000, n_mels=128, hop_length=512):
    y, _ = librosa.load(audio_path, sr=sr)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, hop_length=hop_length)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    return mel_db.T  # [time, freq]

In [ ]:
file_id = "00_BN1-129-Eb_comp_mic"

jams_path = os.path.join(ANNOTATION_DIR, file_id.replace("_mic", "") + ".jams")
audio_path = os.path.join(AUDIO_DIR, file_id + ".wav")

notes_df = extract_note_events(jams_path)
mel = extract_melspec(audio_path)

print("🎼 Notalar:")
print(notes_df.head())

print("\n🎧 MelSpectrogram shape:", mel.shape)

🎼 Notalar:
       onset  duration      pitch
0   7.457274  0.464399  44.018917
1   7.923578  0.922993  44.224918
2   8.870789  0.191565  44.106065
3  14.834689  0.487619  46.053185
4  15.333215  1.416417  46.071249

🎧 MelSpectrogram shape: (698, 128)


In [ ]:
def get_all_ids(annotation_dir):
    file_ids = []
    for filename in os.listdir(annotation_dir):
        if filename.endswith(".jams"):
            file_ids.append(filename.replace(".jams", ""))
    return sorted(file_ids)

In [ ]:
all_data = []

file_ids = get_all_ids(ANNOTATION_DIR)

for file_id in tqdm(file_ids):
    jams_path = os.path.join(ANNOTATION_DIR, file_id + ".jams")
    audio_path = os.path.join(AUDIO_DIR, file_id + "_mic.wav")

    if not os.path.exists(audio_path):
        print(f"⚠️ {file_id} için audio dosyası bulunamadı, atlanıyor.")
        continue

    try:
        mel = extract_melspec(audio_path)  # [T x F]
        notes_df = extract_note_events(jams_path)

        all_data.append({
            "id": file_id,
            "mel": mel,
            "notes": notes_df
        })
    except Exception as e:
        print(f"❌ {file_id} yüklenirken hata oluştu: {e}")

100%|██████████| 360/360 [06:51<00:00,  1.14s/it]


In [ ]:
print(f"Toplam dosya sayısı: {len(all_data)}")
print("Örnek dosya ID:", all_data[0]['id'])
print("Spectrogram shape:", all_data[0]['mel'].shape)
print("İlk 5 nota:")
print(all_data[0]['notes'].head())

Toplam dosya sayısı: 360
Örnek dosya ID: 00_BN1-129-Eb_comp
Spectrogram shape: (698, 128)
İlk 5 nota:
       onset  duration      pitch
0   7.457274  0.464399  44.018917
1   7.923578  0.922993  44.224918
2   8.870789  0.191565  44.106065
3  14.834689  0.487619  46.053185
4  15.333215  1.416417  46.071249


## **Frame Labels**

In [ ]:
OUTPUT_DIR = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/processed_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
def generate_frame_labels(note_df, n_frames, fps=31.25, pitch_min=40, pitch_max=88):
    """
    MelSpectrogram uzunluğuna tam uyumlu frame-level pitch label matrisi üretir.

    Parameters:
        note_df: DataFrame (onset, duration, pitch)
        n_frames: MelSpectrogram'daki toplam frame sayısı
        fps: Frame/saniye (örnek: 16000 / 512 = 31.25)
        pitch_min: Alt sınır (gitar için genellikle 40)
        pitch_max: Üst sınır (genelde 88 - piyano sınırı)

    Returns:
        label_matrix: np.array [n_frames, n_pitches] (binary multi-label matris)
    """
    n_pitches = pitch_max - pitch_min + 1
    label_matrix = np.zeros((n_frames, n_pitches), dtype=np.float32)

    for _, row in note_df.iterrows():
        pitch = int(round(row['pitch']))
        if pitch < pitch_min or pitch > pitch_max:
            continue

        onset_frame = int(np.floor(row['onset'] * fps))
        offset_frame = int(np.ceil((row['onset'] + row['duration']) * fps))

        # Güvenlik: frame sayısını aşmasın
        onset_frame = max(0, onset_frame)
        offset_frame = min(n_frames, offset_frame)

        label_matrix[onset_frame:offset_frame, pitch - pitch_min] = 1.0

    return label_matrix

In [ ]:
fps = 16000 / 512  # 31.25

for sample in tqdm(all_data):
    file_id = sample["id"]
    mel = sample["mel"]
    note_df = sample["notes"]
    n_frames = mel.shape[0]

    try:
        label_matrix = generate_frame_labels(
            note_df,
            n_frames=n_frames,
            fps=fps,
            pitch_min=40,
            pitch_max=88
        )

        # ✅ Doğrudan Drive'daki hedef klasöre kayıt
        save_path = os.path.join(OUTPUT_DIR, f"{file_id}.npz")
        np.savez_compressed(save_path, mel=mel, label=label_matrix)

    except Exception as e:
        print(f"❌ {file_id} işlenirken hata: {e}")

100%|██████████| 360/360 [00:20<00:00, 17.60it/s]


In [ ]:
sample_file = os.listdir(OUTPUT_DIR)[0]
data = np.load(os.path.join(OUTPUT_DIR, sample_file))

mel = data['mel']
label = data['label']

print("Mel shape:", mel.shape)
print("Label shape:", label.shape)
print("Etkin frame sayısı:", (label.sum(axis=1) > 0).sum())

Mel shape: (698, 128)
Label shape: (698, 49)
Etkin frame sayısı: 630
